In [4]:
import numpy as np
import matplotlib.pyplot as plt

#The file 'OnlineNewsPopularity.csv' from https://archive.ics.uci.edu/dataset/332/online+news+popularity is required
file_dir = 'OnlineNewsPopularity.csv'
columns = []
for x in range(61):
    if x == 0 or x == 1:
        continue
    else:
        columns = columns + [x]

data = np.loadtxt(file_dir, delimiter = ',', skiprows=1, usecols = columns)
np.savetxt('Articles_Data.csv', data, delimiter=',')

In [5]:
# Loading the data with target vector
file_dir = 'Articles_Data.csv'
data = np.loadtxt(file_dir, delimiter=',')
avg_std = np.empty((59,2), dtype = float)

print(data.shape)

# Standardizing features and target vector
for col in range(58):
    feature_vector = data[0:39644,col]

    average = np.average(feature_vector)
    std = np.std(feature_vector)

    avg_std[col][0] = average
    avg_std[col][1] = std

    stand_vector = (feature_vector - average) / std
    for row in range(39644):
        data[row,col] = stand_vector[row]

np.savetxt('Articles_Stand_Data.csv', data, delimiter=',')

(39644, 59)


In [6]:
from sklearn.model_selection import KFold

# Loading standardized data with target vector
file_dir = 'Articles_Stand_Data.csv'
data = np.loadtxt(file_dir, delimiter=',')

# Separating data into features matrix and target vector
dat_stand = data[:,:]
dat_noTarget = dat_stand[:,0:58]
dat_Target = dat_stand[:,58]

In [8]:
from scipy.optimize import curve_fit
# Separate data into folds
folds = 5
kf = KFold(n_splits=10)
kf.get_n_splits(dat_noTarget.shape)

# Define non-linear models
# Inverse tangent model
def fun(xdata, x):
  F = np.zeros(xdata.shape[0])
  for i in range(xdata.shape[0]):
    F[i] = np.arctan(xdata[i,:] * np.transpose(x))
  return F

# Initial parameters guess
p0 = np.ones(58)

# Iterating
for (train_index, test_index) in kf.split(dat_noTarget):
  # Separating training and testing set for each fold
  X_train = dat_noTarget[train_index,:]
  y_train = dat_Target[train_index]
  X_test = dat_noTarget[test_index,:]
  y_test = dat_Target[test_index]

  # Run non-linear fitting algorithm
  parameters = curve_fit(fun, X_train, y_train, p0)

TypeError: fun() takes 2 positional arguments but 59 were given